In [1]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# # multi gpu code
# from keras.utils.training_utils import multi_gpu_model
# model=multi_gpu_model(model,gpus=2)

In [3]:
# python internal
import os
from glob import glob

# matplotlib
import matplotlib
%matplotlib inline
matplotlib.rcParams['image.cmap'] = 'gray'
matplotlib.rcParams['font.size'] = 20
import matplotlib.pyplot as plt

# data processing
import numpy as np
import pandas as pd
import cv2
from skimage.io import *
from skimage.color import *
from skimage.transform import *

# deep learning
from keras.models import Sequential, Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Flatten, Dense
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Visualization
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

C:\Users\snu\AppData\Local\Continuum\anaconda3\envs\venv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
def imshowx(img,h,w):
    fig, ax=plt.subplots(h,w,figsize=(4*w,4*h))
    fig.set_facecolor('white')
    ax=ax.ravel()
    for i,a in enumerate(ax):
        a.axis('off')
        if i<img.shape[0]:
            a.imshow(img[i])
    plt.show()

def overlay(img,mask):
    z=np.zeros_like(img)
    return cv2.addWeighted(np.dstack((img,img,img)),.5,np.dstack((z,z,mask)),.5,0)

seed=42

In [5]:
IMG="data1/l_spine_jpg/"
MASK="../input/제욱new/"
IMG_HEIGHT=IMG_WIDTH=512

seed=42

In [6]:
img_names=[os.path.split(g)[1] for g in glob(IMG+"*")]
mask_names=[os.path.split(g)[1] for g in glob(MASK+"*")]

In [7]:
img_names, mask_names

([],
 ['data1-L-001.png',
  'data1-L-002.png',
  'data1-L-003.png',
  'data1-L-004.png',
  'data1-L-005.png',
  'data1-L-006.png',
  'data1-L-007.png',
  'data1-L-008.png',
  'data1-L-009.png',
  'data1-L-010.png',
  'data1-L-011.png',
  'data1-L-012.png',
  'data1-L-013.png',
  'data1-L-014.png',
  'data1-L-015.png',
  'data1-L-016.png',
  'data1-L-017.png',
  'data1-L-018.png',
  'data1-L-019.png',
  'data1-L-020.png',
  'data1-L-021.png',
  'data1-L-022.png',
  'data1-L-023.png',
  'data1-L-024.png',
  'data1-L-026.png',
  'data1-L-027.png',
  'data1-L-028.png',
  'data1-L-029.png',
  'data1-L-030.png',
  'data1-L-031.png',
  'data1-L-032.png',
  'data1-L-033.png',
  'data1-L-035.png',
  'data1-L-036.png',
  'data1-L-037.png',
  'data1-L-038.png',
  'data1-L-039.png',
  'data1-L-040.png',
  'data1-L-041.png',
  'data1-L-042.png',
  'data1-L-043.png',
  'data1-L-044.png',
  'data1-L-045.png',
  'data1-L-046.png',
  'data1-L-047.png',
  'data1-L-048.png',
  'data1-L-049.png',
  'data1

In [8]:
gl=[MASK+f for f in names]
print(gl)

NameError: name 'names' is not defined

In [ ]:
# gl=glob(MASK+"*")

total=len(gl)
img=np.zeros((total,IMG_HEIGHT,IMG_WIDTH,1))
mask=np.zeros((total,IMG_HEIGHT,IMG_WIDTH,1))

for i,g in enumerate(gl):
    p,f=os.path.split(g)
    
    img[i,:,:,0]=resize(rgb2gray(imread(IMG+f)),
                        (IMG_HEIGHT,IMG_WIDTH),
                        anti_aliasing=True,
                        mode='reflect')
    
    mask[i,:,:,0]=resize(rgb2gray(imread(g))!=0,
                         (IMG_HEIGHT,IMG_WIDTH),
                         anti_aliasing=False, # anti_aliasing = false
                         mode='reflect') 

from sklearn.model_selection import train_test_split
img_train,img_val,mask_train,mask_val=train_test_split(img,mask,test_size=.1,random_state=seed)

In [ ]:
img_train=np.array(img_train)
mask_train=np.array(mask_train)

In [ ]:
img_train.shape,mask_train.shape

In [ ]:
imshowx(np.array([overlay(img_train[i],mask_train[i]) for i in range(16)]),4,4)

In [ ]:
data_gen_args= dict(
    rotation_range=45,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.7,
    zoom_range=[.9,2.2],
    horizontal_flip=True,
    fill_mode='nearest')

img_train_datagen=ImageDataGenerator(**data_gen_args)
mask_train_datagen=ImageDataGenerator(**data_gen_args)

img_train_generator=img_train_datagen.flow(img_train,batch_size=16,seed=seed)
mask_train_generator=mask_train_datagen.flow(np.repeat(mask_train,16,axis=0),batch_size=16,seed=seed)

train_generator=zip(img_train_generator,mask_train_generator)

# remove validation augmentation
img_val_datagen=ImageDataGenerator()
mask_val_datagen=ImageDataGenerator()

img_val_generator=img_val_datagen.flow(np.repeat(img_val,6,axis=0),batch_size=16,seed=seed)
mask_val_generator=mask_val_datagen.flow(np.repeat(mask_val,6,axis=0),batch_size=16,seed=seed)
                                          
val_generator=zip(img_val_generator,mask_val_generator)

In [ ]:
# train sanity check
img_preview,mask_preview=next(train_generator)
imshowx(np.array([overlay(img_preview[i],mask_preview[i]) for i in range(16)]),4,4)

# validation sanity check
img_preview,mask_preview=next(val_generator)
imshowx(np.array([overlay(img_preview[i],mask_preview[i]) for i in range(16)]),4,4)

In [ ]:
# 남의 코드
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [ ]:
ls "../../Downloads/png/png/"